In [ ]:
from src import *

In [ ]:
survey = pd.read_excel('./data/raw/NARIG_Survey.xlsx', usecols='A:EG')
column_names = pd.read_json('./data/processed/json/column_names.json', typ='series')

survey.rename(columns=column_names, inplace=True)
survey.replace(replace_map, inplace=True)
survey.replace(ordinal_map, inplace=True)

survey['age'] = survey['age'].replace(0, np.nan)
survey['gender'] = survey['gender'].str.split().str[-1]

orig_col_order = list(survey.columns)